<a href="https://colab.research.google.com/github/branjbar/nlp-bert-family/blob/master/pytorch/pythorch_roberta_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# To avoid notebook get disconnected
"""
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect,60000)
"""

"""
caffeinate
"""

'\ncaffeinate\n'

In [0]:
# TODOS:
#X add tic and toc to calculate the total time!
## check the pretrained model
#X remove neutral from the filtering
## check the configuraion values here: https://www.youtube.com/watch?v=2oWf4v6QEV8

In [0]:
from google.colab import drive
drive.mount('/content/drive')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Wed Apr 29 12:41:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    46W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+--------------------------------------

In [0]:
!pip install transformers
import numpy as np
import pandas as pd
import os
import tokenizers
import string
import torch
import transformers
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import re
from sklearn import model_selection
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import time

In [0]:
KAGGLE_INPUT_PATH = "../input/"
COLAB_INPUT_PATH = "/content/drive/My Drive/input/"
INPUT_PATH = COLAB_INPUT_PATH

DATA_PATH = INPUT_PATH + "tweet-sentiment-extraction/"
CHECKPOINT_PATH = INPUT_PATH + "model_checkpoint/roberta/"
CHECKPOINT_PATH_TORCH = INPUT_PATH + "model_checkpoint/roberta/torch/"
ROBERTA_PATH = INPUT_PATH + 'models/output_twitter/'
# ROBERTA_PATH = INPUT_PATH + 'models/roberta-base/'

VER='pytorch-v2'

In [0]:
def print_to_file(text):
  try:
    f = open(CHECKPOINT_PATH_TORCH + VER + "-logs.txt", 'a')
  except:
    f = open(CHECKPOINT_PATH_TORCH + VER + "-logs.txt", 'w')

  print(text)
  f.write(time.strftime("%Y%m%d-%H%M%S"))
  f.write("  -- ")
  f.write(text)
  f.write("\n")

class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        if self.name:
            print_to_file('[%s]' % self.name,)
        print_to_file('Elapsed: %.2fmin' % ((time.time() - self.tstart)/60))

def jaccard(str1, str2): 

    a = set(str(str1).lower().split()) 
    b = set(str(str2).lower().split())

    c = a.intersection(b)

    if (len(a) + len(b) - len(c)) > 0:
      return float(len(c)) / (len(a) + len(b) - len(c))
    else:
      return 0

In [0]:
########## CONFIG #####################
class config():
    """
    basic configuration of the model/training
    """
    MAX_LEN = 192
    TRAIN_BATCH_SIZE = 32
    VALID_BATCH_SIZE = 8
    EPOCHS = 3
    DEVICE = torch.device("cuda")
    TOKENIZER = tokenizers.ByteLevelBPETokenizer(
        vocab_file=f"{ROBERTA_PATH}/vocab.json", 
        merges_file=f"{ROBERTA_PATH}/merges.txt", 
        lowercase=True,
        add_prefix_space=True
        )

In [0]:
class TweetModel(transformers.BertPreTrainedModel):
    def __init__(self, conf):
        super(TweetModel, self).__init__(conf)
        self.roberta = transformers.RobertaModel.from_pretrained(ROBERTA_PATH, config=conf)
        self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768 * 2, 2)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
    
    def forward(self, ids, mask, token_type_ids):
        _, _, out = self.roberta(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )

        out = torch.cat((out[-1], out[-2]), dim=-1)
        out = self.drop_out(out)
        logits = self.l0(out)

        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

In [0]:
def process_data(tweet, selected_text, sentiment, tokenizer, max_len):
    tweet = " " + " ".join(str(tweet).split())
    selected_text = " " + " ".join(str(selected_text).split())

    len_st = len(selected_text) - 1
    idx0 = None
    idx1 = None

    for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
        if " " + tweet[ind: ind+len_st] == selected_text:
            idx0 = ind
            idx1 = ind + len_st - 1
            break

    char_targets = [0] * len(tweet)
    if idx0 != None and idx1 != None:
        for ct in range(idx0, idx1 + 1):
            char_targets[ct] = 1
    
    tok_tweet = tokenizer.encode(tweet)
    input_ids_orig = tok_tweet.ids
    tweet_offsets = tok_tweet.offsets
    
    target_idx = []
    for j, (offset1, offset2) in enumerate(tweet_offsets):
        if sum(char_targets[offset1: offset2]) > 0:
            target_idx.append(j)
    
    targets_start = target_idx[0]
    targets_end = target_idx[-1]

    sentiment_id = {
        'positive': 1313,
        'negative': 2430,
        'neutral': 7974
    }
    
    input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_orig + [2]
    token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_orig) + 1)
    mask = [1] * len(token_type_ids)
    tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
    targets_start += 4
    targets_end += 4

    padding_length = max_len - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([1] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
    return {
        'ids': input_ids,
        'mask': mask,
        'token_type_ids': token_type_ids,
        'targets_start': targets_start,
        'targets_end': targets_end,
        'orig_tweet': tweet,
        'orig_selected': selected_text,
        'sentiment': sentiment,
        'offsets': tweet_offsets
    }


class TweetDataset:
    def __init__(self, tweet, sentiment, selected_text):
        self.tweet = tweet
        self.sentiment = sentiment
        self.selected_text = selected_text
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
    
    def __len__(self):
        return len(self.tweet)

    def __getitem__(self, item):
        data = process_data(
            self.tweet[item], 
            self.selected_text[item], 
            self.sentiment[item],
            self.tokenizer,
            self.max_len
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long),
            'targets_start': torch.tensor(data["targets_start"], dtype=torch.long),
            'targets_end': torch.tensor(data["targets_end"], dtype=torch.long),
            'orig_tweet': data["orig_tweet"],
            'orig_selected': data["orig_selected"],
            'sentiment': data["sentiment"],
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }

In [0]:
def calculate_jaccard_score(
    original_tweet, 
    target_string, 
    sentiment_val, 
    idx_start, 
    idx_end, 
    offsets,
    verbose=False):
    
    if idx_end < idx_start:
        idx_end = idx_start
    
    filtered_output  = ""
    for ix in range(idx_start, idx_end + 1):
        filtered_output += original_tweet[offsets[ix][0]: offsets[ix][1]]
        if (ix+1) < len(offsets) and offsets[ix][1] < offsets[ix+1][0]:
            filtered_output += " "

    if len(original_tweet.split()) < 2:
        filtered_output = original_tweet

    if sentiment_val != "neutral" and verbose == True:
        if filtered_output.strip().lower() != target_string.strip().lower():
            print("********************************")
            print(f"Output= {filtered_output.strip()}")
            print(f"Target= {target_string.strip()}")
            print(f"Tweet= {original_tweet.strip()}")
            print("********************************")

    jac = jaccard(target_string.strip(), filtered_output.strip())
    return jac, filtered_output

In [0]:
def load_data(train_df, train_idx, valid_idx):
    df_train = train_df.iloc[train_idx]
    df_valid = train_df.iloc[valid_idx]

    train_dataset = TweetDataset(
        tweet=df_train.text.values,
        sentiment=df_train.sentiment.values,
        selected_text=df_train.selected_text.values
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=4
    )

    valid_dataset = TweetDataset(
        tweet=df_valid.text.values,
        sentiment=df_valid.sentiment.values,
        selected_text=df_valid.selected_text.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=2
    )

    return train_data_loader, valid_data_loader, df_train, df_valid
        
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)
    return total_loss

def calculate_valid_score(model, valid_data_loader):
    with torch.no_grad():
        model.eval()
        final_output = []
        jaccard_scores = []

        for d in tqdm(valid_data_loader, total=len(valid_data_loader), position=0, leave=True):
            outputs_start, outputs_end = model(
                ids=d["ids"].to(config.DEVICE, dtype=torch.long),
                mask=d["mask"].to(config.DEVICE, dtype=torch.long),
                token_type_ids=d["token_type_ids"].to(config.DEVICE, dtype=torch.long)
            )
            outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
            outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()

            for px in range(len(d["offsets"])):

                targets_start = d["targets_start"] [px]
                targets_end = d["targets_end"][px]
                offsets = d["offsets"][px]
                
                
                jac, output_sentence = calculate_jaccard_score(
                    original_tweet=d["orig_tweet"][px],
                    target_string=d["orig_selected"][px],
                    sentiment_val=d["sentiment"][px],
                    idx_start=np.argmax(outputs_start[px, :]),
                    idx_end=np.argmax(outputs_end[px, :]),
                    offsets=d["offsets"][px]
                )
                final_output.append(output_sentence)
                jaccard_scores.append(jac)
        return np.mean(jaccard_scores), jaccard_scores, final_output

def train_model(train_data_loader, fold_num, num_epochs, num_train_steps, train_jac_metric=False):

    model = TweetModel(conf=model_config).to(config.DEVICE)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )
    model.zero_grad()
    best_score = -100
    for epoch in range(num_epochs):
        model.train()
        print_to_file("Fold %d, Epoch %d" % (fold_num+1, epoch+1))
        time.sleep(1) 
        fold_jaccard_scores = []
        for d in tqdm(train_data_loader, total=len(train_data_loader), position=0, leave=True):
            
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            sentiment = d["sentiment"]
            orig_selected = d["orig_selected"]
            orig_tweet = d["orig_tweet"]
            targets_start = d["targets_start"]
            targets_end = d["targets_end"]
            offsets = d["offsets"]

            ids = ids.to(config.DEVICE, dtype=torch.long)
            token_type_ids = token_type_ids.to(config.DEVICE, dtype=torch.long)
            mask = mask.to(config.DEVICE, dtype=torch.long)
            targets_start = targets_start.to(config.DEVICE, dtype=torch.long)
            targets_end = targets_end.to(config.DEVICE, dtype=torch.long)

            model.zero_grad()
            outputs_start, outputs_end = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids,
            )
            loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            if train_jac_metric:
                outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
                outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()

                for px, tweet in enumerate(orig_tweet):
                    selected_tweet = orig_selected[px]
                    tweet_sentiment = sentiment[px]
                    jac, output_sentence = calculate_jaccard_score(
                        original_tweet=tweet,
                        target_string=selected_tweet,
                        sentiment_val=tweet_sentiment,
                        idx_start=np.argmax(outputs_start[px, :]),
                        idx_end=np.argmax(outputs_end[px, :]),
                        offsets=offsets[px]
                    )
                    fold_jaccard_scores.append(jac)
                print_to_file("Jaccard average - train: " + str(np.mean(f_jaccard_scores)))

        valid_jaccard_score, oof_jaccard_scores, oof_final_output = calculate_valid_score(model, valid_data_loader)

        print_to_file("Jaccard average - valid: " + str(valid_jaccard_score)) 
        if valid_jaccard_score > best_score:
            best_score = valid_jaccard_score
            torch.save(model.state_dict(), CHECKPOINT_PATH_TORCH + '%s-roberta-%i.bin'%(VER,fold_num+1))
        else:
            print_to_file("Jaccard average didn't improve")

    return best_score, model, oof_jaccard_scores, oof_final_output

In [0]:
train_df = pd.read_csv(DATA_PATH + 'train.csv').dropna().reset_index()#.sample(n=300).reset_index()

df_test = pd.read_csv(DATA_PATH + 'test.csv')
df_test.loc[:, "selected_text"] = df_test.text.values

In [0]:
model_config = transformers.RobertaConfig.from_pretrained(ROBERTA_PATH)
model_config.output_hidden_states = True

In [0]:
num_folds = 5
num_epochs = 3
jaccard_total = []

kfold = model_selection.KFold(n_splits=num_folds, shuffle=True, random_state=42)
for fold_num, (train_idx, valid_idx) in enumerate(kfold.split(train_df.text)):
    with Timer():
        print_to_file("Fold %i" % (fold_num+1))
        train_data_loader, valid_data_loader, df_train, df_valid = load_data(train_df, train_idx, valid_idx)
        num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
        
        if not os.path.exists(CHECKPOINT_PATH_TORCH + '%s-roberta-%i.bin'%(VER,fold_num+1)):
            
            best_score, _, oof_jaccard_scores, oof_final_output = train_model(train_data_loader, fold_num, num_epochs, num_train_steps)
        else:
            print_to_file("Fold %d, Loading Model" % (fold_num+1))
            model = TweetModel(conf=model_config).to(config.DEVICE)
            model.load_state_dict(torch.load(CHECKPOINT_PATH_TORCH + '%s-roberta-%i.bin'%(VER,fold_num+1)))
            model.eval()

            best_score, oof_jaccard_scores, oof_final_output = calculate_valid_score(model, valid_data_loader)

            print_to_file("Jaccard average - valid: " + str(best_score))

        train_df.loc[valid_idx, "jac"] = oof_jaccard_scores
        train_df.loc[valid_idx, "pred"] = oof_final_output
        jaccard_total.append(best_score)

cv_score = np.mean(jaccard_total)
print_to_file("==================================")
print_to_file("CV jaccard score: " + str(cv_score))
print_to_file("==================================")

Fold 1
Fold 1, Loading Model


100%|██████████| 687/687 [00:39<00:00, 17.45it/s]


Jaccard average - valid: 0.7082862069268179
Elapsed: 0.92min
Fold 2
Fold 2, Loading Model


100%|██████████| 687/687 [00:39<00:00, 17.39it/s]


Jaccard average - valid: 0.7015439676910558
Elapsed: 0.77min
Fold 3
Fold 3, Epoch 1


  0%|          | 0/687 [00:00<?, ?it/s]/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)
 39%|███▉      | 268/687 [02:37<04:05,  1.71it/s]

Elapsed: 2.74min


KeyboardInterrupt: ignored

In [0]:
%%capture

model1 = TweetModel(conf=model_config)
model1.to(config.DEVICE)
model1.load_state_dict(torch.load(CHECKPOINT_PATH_TORCH + '%s-roberta-%i.bin'%(VER,1)))
model1.eval()

model2 = TweetModel(conf=model_config)
model2.to(config.DEVICE)
model2.load_state_dict(torch.load(CHECKPOINT_PATH_TORCH + '%s-roberta-%i.bin'%(VER,2)))
model2.eval()

model3 = TweetModel(conf=model_config)
model3.to(config.DEVICE)
model3.load_state_dict(torch.load(CHECKPOINT_PATH_TORCH + '%s-roberta-%i.bin'%(VER,3)))
model3.eval()

model4 = TweetModel(conf=model_config)
model4.to(config.DEVICE)
model4.load_state_dict(torch.load(CHECKPOINT_PATH_TORCH + '%s-roberta-%i.bin'%(VER,4)))
model4.eval()

model5 = TweetModel(conf=model_config)
model5.to(config.DEVICE)
model5.load_state_dict(torch.load(CHECKPOINT_PATH_TORCH + '%s-roberta-%i.bin'%(VER,5)))
model5.eval()

In [0]:
test_dataset = TweetDataset(
        tweet=df_test.text.values,
        sentiment=df_test.sentiment.values,
        selected_text=df_test.selected_text.values
    )

data_loader = torch.utils.data.DataLoader(
    test_dataset,
    shuffle=False,
    batch_size=config.VALID_BATCH_SIZE,
    num_workers=1
)

final_output = []
with torch.no_grad():
    for d in tqdm(data_loader, total=len(data_loader), position=0, leave=True):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        sentiment = d["sentiment"]
        orig_selected = d["orig_selected"]
        orig_tweet = d["orig_tweet"]
        targets_start = d["targets_start"]
        targets_end = d["targets_end"]
        offsets = d["offsets"].numpy()

        ids = ids.to(config.DEVICE, dtype=torch.long)
        token_type_ids = token_type_ids.to(config.DEVICE, dtype=torch.long)
        mask = mask.to(config.DEVICE, dtype=torch.long)
        targets_start = targets_start.to(config.DEVICE, dtype=torch.long)
        targets_end = targets_end.to(config.DEVICE, dtype=torch.long)

        outputs_start1, outputs_end1 = model1(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )
        
        outputs_start2, outputs_end2 = model2(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )
        
        outputs_start3, outputs_end3 = model3(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )
        
        outputs_start4, outputs_end4 = model4(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )
        
        outputs_start5, outputs_end5 = model5(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )
        outputs_start = (outputs_start1 + outputs_start2 + outputs_start3 + outputs_start4 + outputs_start5) / 5
        outputs_end = (outputs_end1 + outputs_end2 + outputs_end3 + outputs_end4 + outputs_end5) / 5
        
        outputs_start = torch.softmax(outputs_start, dim=1).cpu().detach().numpy()
        outputs_end = torch.softmax(outputs_end, dim=1).cpu().detach().numpy()
        jaccard_scores = []
        for px, tweet in enumerate(orig_tweet):
            selected_tweet = orig_selected[px]
            tweet_sentiment = sentiment[px]
            _, output_sentence = calculate_jaccard_score(
                original_tweet=tweet,
                target_string=selected_tweet,
                sentiment_val=tweet_sentiment,
                idx_start=np.argmax(outputs_start[px, :]),
                idx_end=np.argmax(outputs_end[px, :]),
                offsets=offsets[px]
            )
            final_output.append(output_sentence)

In [0]:
df_test['selected_text'] = final_output

df_test['selected_text'] = df_test['selected_text'].apply(lambda x: x.replace('!!!!', '!') if len(x.split())==1 else x)
df_test['selected_text'] = df_test['selected_text'].apply(lambda x: x.replace('..', '.') if len(x.split())==1 else x)
df_test['selected_text'] = df_test['selected_text'].apply(lambda x: x.replace('...', '.') if len(x.split())==1 else x)

df_test[['textID','selected_text']].to_csv(DATA_PATH + 'submission-%s.csv'%VER, index=False)